### Test multi-lingual model

In [1]:
import torch
from torch import nn

# HuggingFace transformers (availale in OCI DS conda nlp env)
# see: https://github.com/huggingface/transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
#
# a more flexible version
#
class MultiSentimentAnalyzer:
    # load the tokenizer and transformer
    def __init__(self, MODEL_NAME, labels):

        #
        # attribute definitions
        #

        # for rounding the scores
        self._DEC_DIGITS = 4

        # name of HuggingFace model used
        self._MODEL_NAME = MODEL_NAME

        # the list of defined labels (and therefore we will have 5 scores)
        # this is changed from ITA class
        self._LABELS = labels

        print("Loading model...")
        self.tokenizer = AutoTokenizer.from_pretrained(self._MODEL_NAME)
        # Load the model
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self._MODEL_NAME
        )

        print("Model loading completed!")

    #
    # does the scoring on a single sentence a time
    #
    def score(self, input_sentence):
        # encode the sentence and create the input tensor (in PyTorch format)
        input_ids = self.tokenizer(
            input_sentence, add_special_tokens=True, return_tensors="pt"
        )["input_ids"]

        # output from tokenizer is already a tensor

        # Call the model and get the logits
        with torch.no_grad():
            logits = self.model(input_ids)["logits"]

        # The model was trained with a Log Likelyhood + Softmax combined loss, hence to extract probabilities we need a softmax on top of the logits tensor
        proba = nn.functional.softmax(logits, dim=1)

        # to remove the added dimension with squeeze
        # proba is a tuple () with one value for each label
        scores = proba.squeeze(0)

        # get rid of tensor and round and
        # prepare the output json

        ret_vet = []

        for i, label in enumerate(self._LABELS):
            ret_vet.append({"label": label, "score": self.round(scores[i])})

        return ret_vet

    # only to format output
    def format_scores(self, scores):
        score_str = ""

        for v_score in scores:
            score_str += str(v_score["label"]) + ": " + str(v_score["score"]) + "|"

        return score_str

    # utility to get rid of tensor and round
    def round(self, tens_val):
        return round(tens_val.item(), self._DEC_DIGITS)

In [3]:
%%time

# loading the model: pass the HF model name
MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"

# labels are dependent on the used mdel , see HF documentation
sent_analyzer = MultiSentimentAnalyzer(
    MODEL_NAME, labels=["1 star", "2 star", "3 star", "4 star", "5 star"]
)

Loading model...
Model loading completed!
CPU times: user 1.21 s, sys: 237 ms, total: 1.45 s
Wall time: 6.28 s


### Some tests

In [5]:
%%time

scores = sent_analyzer.score("Il servizio offerto alla clientela è ottimo")

sent_analyzer.format_scores(scores)

CPU times: user 135 ms, sys: 6.77 ms, total: 141 ms
Wall time: 36 ms


'1 star: 0.0056|2 star: 0.0059|3 star: 0.0518|4 star: 0.323|5 star: 0.6137|'

In [6]:
scores = sent_analyzer.score(
    "The service you have offered to your customers is really good"
)

sent_analyzer.format_scores(scores)

'1 star: 0.0031|2 star: 0.0043|3 star: 0.0572|4 star: 0.4137|5 star: 0.5217|'

In [7]:
%%time

input_sentences = [
    "E' un prodotto pessimo",
    "La sua organizzazione ha fornito un buon servizio alla clientela",
    "La sua organizzazione non ha fornito un buon servizio alla clientela",
    "Non credo che la sua organizzazione abbia fornito un buon servizio alla clientela",
    "Il prodotto non funziona, non comprero' più nulla dalla vostra azienda",
    "Io penso che la sua organizzazione non abbia fornito un buon servizio alla clientela",
    "La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona?",
    "La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona",
    "La vostra organizzazione offre servizi pessimi",
    "La vostra organizzazione offre servizi non adeguati",
    "Sono molto soddisfatto del tuo lavoro",
    "non sono del tutto sicuro che il lavoro sia adeguato",
    "l'azienda dovrebbe offrire servizi migliori",
    "il supporto offerto dal customer care non è stato adeguato",
    "il risultato è pessimo",
    "il Napoli ha giocato una partita decente",
    "il lavoro dell'allenatore è stato modesto",
]


# object already instantiated

for i, sentence in enumerate(input_sentences):

    #
    # here I do the scoring on a single sentence
    #
    scores = sent_analyzer.score(sentence)

    # scores is a dict of []

    print(f"{i+1}. {sentence}")
    print(sent_analyzer.format_scores(scores))
    print()

# formatting
print()

1. E' un prodotto pessimo
1 star: 0.9365|2 star: 0.0558|3 star: 0.0063|4 star: 0.0008|5 star: 0.0006|

2. La sua organizzazione ha fornito un buon servizio alla clientela
1 star: 0.0041|2 star: 0.0072|3 star: 0.1284|4 star: 0.5368|5 star: 0.3235|

3. La sua organizzazione non ha fornito un buon servizio alla clientela
1 star: 0.2703|2 star: 0.4699|3 star: 0.2328|4 star: 0.0228|5 star: 0.0042|

4. Non credo che la sua organizzazione abbia fornito un buon servizio alla clientela
1 star: 0.0336|2 star: 0.0579|3 star: 0.2817|4 star: 0.4109|5 star: 0.2159|

5. Il prodotto non funziona, non comprero' più nulla dalla vostra azienda
1 star: 0.9054|2 star: 0.0825|3 star: 0.0105|4 star: 0.0009|5 star: 0.0006|

6. Io penso che la sua organizzazione non abbia fornito un buon servizio alla clientela
1 star: 0.304|2 star: 0.4361|3 star: 0.2269|4 star: 0.0261|5 star: 0.0068|

7. La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona?
1 star: 0.1193|2 star: 0.2377|

### custom test

In [8]:
sentence = "L'organizzazione di questa riunione è stata ottima"

sent_analyzer.score(sentence)

[{'label': '1 star', 'score': 0.0064},
 {'label': '2 star', 'score': 0.0083},
 {'label': '3 star', 'score': 0.0742},
 {'label': '4 star', 'score': 0.3856},
 {'label': '5 star', 'score': 0.5255}]